In [1]:
import cv2 as cv
import pandas as pd
import numpy as np
import mediapipe as mp

mp_pose = mp.solutions.pose
pose_detector = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.9, min_tracking_confidence=0.9)
mp_drawing = mp.solutions.drawing_utils



I0000 00:00:1756649909.787118 2101261 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4


In [2]:
thresholds_df = pd.read_csv('pose_thresholds.csv')


In [3]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle


In [4]:
def run_calibration(pose_to_check):
    pose_thresholds = thresholds_df[thresholds_df['pose'] == pose_to_check].set_index('joint')

    angle_definitions = {
        'left_shoulder_angle': (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW),
        'left_elbow_angle': (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
        'right_shoulder_angle': (mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW),
        'right_elbow_angle': (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST),
        'left_hip_angle': (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE),
        'left_knee_angle': (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE),
        'right_hip_angle': (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE),
        'right_knee_angle': (mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE)
    }
    
    GREEN = (0, 255, 0)
    RED = (0, 0, 255)
    BLUE = (255, 0, 0)

    cap = cv.VideoCapture(0)
    if not cap.isOpened():
            return

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break
        image = frame
        image = cv.flip(image, 1)
        image_rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        results = pose_detector.process(image_rgb)
        image = cv.cvtColor(image_rgb, cv.COLOR_RGB2BGR)
        
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS)

            for joint_name, row in pose_thresholds.iterrows():
                if (pose_to_check == "Warrior ii pose" and "knee" in joint_name) or \
                   (pose_to_check == "Vrksasana" and ("knee" in joint_name or "elbow" in joint_name)):
                    continue

                if joint_name in angle_definitions:
                        p1_enum, p2_enum, p3_enum = angle_definitions[joint_name]
                        p1 = [landmarks[p1_enum.value].x, landmarks[p1_enum.value].y]
                        p2 = [landmarks[p2_enum.value].x, landmarks[p2_enum.value].y] 
                        p3 = [landmarks[p3_enum.value].x, landmarks[p3_enum.value].y]
                        
                        current_angle = calculate_angle(p1, p2, p3)
                        min_angle = row['min_angle']
                        max_angle = row['max_angle']
                        joint_coords = tuple(np.multiply(p2, [image.shape[1], image.shape[0]]).astype(int))
                        
                        color = GREEN if min_angle <= current_angle <= max_angle else RED
                        cv.putText(image, f"{int(current_angle)}", (joint_coords[0], joint_coords[1] - 20), cv.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, cv.LINE_AA)
                        cv.circle(image, joint_coords, 10, color, -1)
               

            if pose_to_check == "Warrior ii pose":
                try:
                    BENT_KNEE_RANGE = (85, 135)
                    STRAIGHT_KNEE_RANGE = (165, 180)
                    left_knee_angle = calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y], [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y], [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y])
                    right_knee_angle = calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y], [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y], [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y])
                    
                    if left_knee_angle < right_knee_angle:
                        left_correct = BENT_KNEE_RANGE[0] <= left_knee_angle <= BENT_KNEE_RANGE[1]
                        right_correct = STRAIGHT_KNEE_RANGE[0] <= right_knee_angle <= STRAIGHT_KNEE_RANGE[1]
                    else:
                        left_correct = STRAIGHT_KNEE_RANGE[0] <= left_knee_angle <= STRAIGHT_KNEE_RANGE[1]
                        right_correct = BENT_KNEE_RANGE[0] <= right_knee_angle <= BENT_KNEE_RANGE[1]
                    
                    lk_coords = tuple(np.multiply([landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y], [image.shape[1], image.shape[0]]).astype(int))
                    rk_coords = tuple(np.multiply([landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y], [image.shape[1], image.shape[0]]).astype(int))
                    lk_color = GREEN if left_correct else RED
                    rk_color = GREEN if right_correct else RED

                    cv.circle(image, lk_coords, 10, lk_color, -1)
                    cv.putText(image, f"{int(left_knee_angle)}", (lk_coords[0], lk_coords[1] - 20), cv.FONT_HERSHEY_SIMPLEX, 0.7, lk_color, 2, cv.LINE_AA)
                    cv.circle(image, rk_coords, 10, rk_color, -1)
                    cv.putText(image, f"{int(right_knee_angle)}", (rk_coords[0], rk_coords[1] - 20), cv.FONT_HERSHEY_SIMPLEX, 0.7, rk_color, 2, cv.LINE_AA)
                except:
                    pass
            
            if pose_to_check == "Vrksasana":
                    BENT_ELBOW_RANGE_TREE = (60, 120)
                    left_elbow_angle = calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y], [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y], [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y])
                    right_elbow_angle = calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y], [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y], [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y])
                    left_elbow_correct = BENT_ELBOW_RANGE_TREE[0] <= left_elbow_angle <= BENT_ELBOW_RANGE_TREE[1]
                    right_elbow_correct = BENT_ELBOW_RANGE_TREE[0] <= right_elbow_angle <= BENT_ELBOW_RANGE_TREE[1]
                    le_coords = tuple(np.multiply([landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y], [image.shape[1], image.shape[0]]).astype(int))
                    re_coords = tuple(np.multiply([landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y], [image.shape[1], image.shape[0]]).astype(int))
                    le_color = GREEN if left_elbow_correct else RED
                    re_color = GREEN if right_elbow_correct else RED
                    cv.circle(image, le_coords, 10, le_color, -1)
                    cv.circle(image, re_coords, 10, re_color, -1)

                    BENT_KNEE_RANGE_TREE = (30, 100)
                    STRAIGHT_KNEE_RANGE_TREE = (165, 180)
                    left_knee_angle = calculate_angle([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y], [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y], [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y])
                    right_knee_angle = calculate_angle([landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y], [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y], [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y])
                    
                    if landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y < landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y:
                        left_knee_correct = BENT_KNEE_RANGE_TREE[0] <= left_knee_angle <= BENT_KNEE_RANGE_TREE[1]
                        right_knee_correct = STRAIGHT_KNEE_RANGE_TREE[0] <= right_knee_angle <= STRAIGHT_KNEE_RANGE_TREE[1]
                    else:
                        left_knee_correct = STRAIGHT_KNEE_RANGE_TREE[0] <= left_knee_angle <= STRAIGHT_KNEE_RANGE_TREE[1]
                        right_knee_correct = BENT_KNEE_RANGE_TREE[0] <= right_knee_angle <= BENT_KNEE_RANGE_TREE[1]
                    
                    lk_coords = tuple(np.multiply([landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y], [image.shape[1], image.shape[0]]).astype(int))
                    rk_coords = tuple(np.multiply([landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y], [image.shape[1], image.shape[0]]).astype(int))
                    lk_color = GREEN if left_knee_correct else RED
                    rk_color = GREEN if right_knee_correct else RED
                    
                    cv.circle(image, lk_coords, 10, lk_color, -1)
                    cv.circle(image, rk_coords, 10, rk_color, -1)

        cv.putText(image, f"Checking Pose: {pose_to_check}", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, BLUE, 2, cv.LINE_AA)
        cv.imshow('MediaPipe Pose Correction', image)

        if cv.waitKey(5) & 0xFF == 27: 
            break
            
    cap.release()
    cv.destroyAllWindows()

In [5]:
POSE_TO_CHECK = "Dandasana" 
    
run_calibration(POSE_TO_CHECK)

W0000 00:00:1756649910.130251 2103559 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1756649910.174299 2103562 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


W0000 00:00:1756649917.213853 2103558 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
